# TO DO

- [ ] add reward scaling
- [x] override train_agent_batch_with_evaluation
- [ ] support calllable object(hook)
- [ ] change original function to 
 - [ ] return float reward
 - [ ] only one Done
 - [ ] info["needs_reset"]

In [12]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import
from future import standard_library
standard_library.install_aliases()  # NOQA
import argparse
import sys
import logging
from collections import deque

import chainer
from chainer import optimizers
import gym
from gym import spaces
import gym.wrappers
import numpy as np

from chainer import cuda
import chainerrl
from chainerrl.agents.ddpg_ma import DDPG_MA
from chainerrl.agents.ddpg import DDPGModel
from chainerrl import experiments
from chainerrl import explorers
from chainerrl import misc
from chainerrl import policy
from chainerrl import q_functions
from chainerrl import replay_buffer

from flow.multiagent_envs import MultiWaveAttenuationMergePOEnvMeanRew
from flow.scenarios import MergeScenario
from flow.utils.registry import make_create_env

benchmark_name = 'multi_merge'

In [35]:
gpu = None
seed = 0
final_exploration_steps = 10**6
actor_lr = 1e-4
critic_lr = 1e-3
steps = 10 ** 7
n_hidden_channels = 300
n_hidden_layers = 3
replay_start_size = 5000
n_update_times = 1
target_update_interval = 1
target_update_method = 'soft'
soft_update_tau = 1e-2
update_interval = 4
eval_n_runs = 100
eval_interval = 10**5
gamma = 0.995
minibatch_size = 200
use_bn = True
reward_scale_factor = 1e-2
return_window_size = 100
step_offset = 0
log_interval = 5
num_envs = 2
outdir = 'results'

In [24]:
def make_env(create_env):
    def _thunk():
        env = create_env()
        env = chainerrl.wrappers.ScaleReward(env, reward_scale_factor)
        return env
    return _thunk

def make_batch_env(test):
    return chainerrl.envs.MultiprocessVectorEnv(
        [make_env(create_env) for i in range(num_envs)])

In [25]:
benchmark  = __import__(
    "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.mean_rew_flow_params
HORIZON = flow_params['env'].horizon

create_env, env_name = make_create_env(params=flow_params, version=0)

In [58]:
sample_env = create_env()

timestep_limit = flow_params["env"].horizon
obs_size = np.asarray(sample_env.observation_space.shape).prod()
action_space = sample_env.action_space

INFO:root: Starting SUMO on port 57510
DEBUG:root: Cfg file: /headless/flow/flow/core/kernel/scenario/debug/cfg/multi_merge_20190424-1910101556133010.9950006.sumo.cfg
DEBUG:root: Emission file: None
DEBUG:root: Step length: 0.5


In [28]:
action_size = np.asarray(action_space.shape).prod()
if use_bn:
    q_func = q_functions.FCBNLateActionSAQFunction(
        obs_size, action_size,
        n_hidden_channels=n_hidden_channels,
        n_hidden_layers=n_hidden_layers,
        normalize_input=True)
    pi = policy.FCBNDeterministicPolicy(
        obs_size, action_size=action_size,
        n_hidden_channels=n_hidden_channels,
        n_hidden_layers=n_hidden_layers,
        min_action=action_space.low, max_action=action_space.high,
        bound_action=True,
        normalize_input=True)
else:
    q_func = q_functions.FCSAQFunction(
        obs_size, action_size,
        n_hidden_channels=n_hidden_channels,
        n_hidden_layers=n_hidden_layers)
    pi = policy.FCDeterministicPolicy(
        obs_size, action_size=action_size,
        n_hidden_channels=n_hidden_channels,
        n_hidden_layers=n_hidden_layers,
        min_action=action_space.low, max_action=action_space.high,
        bound_action=True)

In [29]:
model = DDPGModel(q_func=q_func, policy=pi)
opt_a = optimizers.Adam(alpha=actor_lr)
opt_c = optimizers.Adam(alpha=critic_lr)
opt_a.setup(model['policy'])
opt_c.setup(model['q_function'])
opt_a.add_hook(chainer.optimizer.GradientClipping(1.0), 'hook_a')
opt_c.add_hook(chainer.optimizer.GradientClipping(1.0), 'hook_c')

rbuf = replay_buffer.ReplayBuffer(5 * 10 ** 5)

In [30]:
ou_sigma = (action_space.high - action_space.low) * 0.2
explorer = explorers.AdditiveOU(sigma=ou_sigma)
agent = DDPG_MA(model, opt_a, opt_c, rbuf, gamma=gamma,
             explorer=explorer, replay_start_size=replay_start_size,
             target_update_method=target_update_method,
             target_update_interval=target_update_interval,
             update_interval=update_interval,
             soft_update_tau=soft_update_tau,
             n_times_update=n_update_times,
             gpu=gpu, minibatch_size=minibatch_size)

In [53]:
env = make_batch_env(False)

INFO:root: Starting SUMO on port 53180
DEBUG:root: Cfg file: /headless/flow/flow/core/kernel/scenario/debug/cfg/multi_merge_20190424-1910101556133010.9950006.sumo.cfg
DEBUG:root: Emission file: None
DEBUG:root: Step length: 0.5
INFO:root: Starting SUMO on port 49462
DEBUG:root: Cfg file: /headless/flow/flow/core/kernel/scenario/debug/cfg/multi_merge_20190424-1910101556133010.9950006.sumo.cfg
DEBUG:root: Emission file: None
DEBUG:root: Step length: 0.5
INFO:root: Starting SUMO on port 49462
INFO:root: Starting SUMO on port 53180
DEBUG:root: Cfg file: /headless/flow/flow/core/kernel/scenario/debug/cfg/multi_merge_20190424-1910101556133010.9950006.sumo.cfg
DEBUG:root: Emission file: None
DEBUG:root: Step length: 0.5
DEBUG:root: Cfg file: /headless/flow/flow/core/kernel/scenario/debug/cfg/multi_merge_20190424-1910101556133010.9950006.sumo.cfg
DEBUG:root: Emission file: None
DEBUG:root: Step length: 0.5


In [32]:
experiments.evaluator.batch_run_evaluation_episodes(env, agent, 1000, None)

[0.22922069449064814,
 0.9044327198833495,
 0.020012175608951334,
 0.0,
 1.0982613590551924,
 0.23122124434649813]

In [46]:
agent.batch_act_and_train(obss)

DEBUG:chainerrl.agents.ddpg:t:12 a:[0.47064388] q:[[-0.22099449]
 [-0.29986876]
 [-0.13345966]]
DEBUG:chainerrl.explorers.additive_ou:t:12 noise:[1.727788]
DEBUG:chainerrl.explorers.additive_ou:t:12 noise:[0.7915134]
DEBUG:chainerrl.explorers.additive_ou:t:12 noise:[0.6915788]
DEBUG:chainerrl.explorers.additive_ou:t:12 noise:[1.2844546]
DEBUG:chainerrl.explorers.additive_ou:t:12 noise:[1.3699714]


[{'flow_1.1': array([0.97069466], dtype=float32),
  'flow_1.2': array([1.8233471], dtype=float32)},
 {'flow_1.0': array([1.8406153], dtype=float32),
  'flow_1.1': array([1.4622778], dtype=float32),
  'flow_1.2': array([0.7869964], dtype=float32)}]

In [34]:
# from IPython.core.debugger import Pdb; Pdb().set_trace()

eval_stats = experiments.eval_performance(
    env=env,
    agent=agent,
    n_steps=None,
    n_episodes=2,
    max_episode_len=timestep_limit)
print('n_runs: {} mean: {} median: {} stdev {}'.format(
    eval_n_runs, eval_stats['mean'], eval_stats['median'],
    eval_stats['stdev']))

n_runs: 100 mean: 0.5216173835616209 median: 0.5216173835616209 stdev 0.737678378202413


In [66]:
sample_env.k.close()

In [48]:
steps = 100
eval_n_steps = 2
eval_interval = 50
experiments.train_agent_batch_with_evaluation(
    agent=agent, env=env, steps=steps,
    eval_env=env, eval_n_steps=None,
    eval_n_episodes=eval_n_runs, eval_interval=eval_interval,
    outdir=outdir)

DEBUG:chainerrl.agents.ddpg:t:0 a:[0.17731178] q:[[-0.2980848 ]
 [-0.13342942]]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[0.16696383]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[-0.08392982]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[-0.22894403]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[-0.4413257]
DEBUG:chainerrl.agents.ddpg:t:4 a:[0.1846306] q:[[-0.2913115 ]
 [-0.12496676]]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[0.19466206]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[0.364954]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[-0.87545013]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[-1.1332892]
DEBUG:chainerrl.agents.ddpg:t:8 a:[0.19499558] q:[[-0.26810604]
 [-0.10397968]]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[-0.9578338]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[-0.31150126]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[-1.0407608]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[-1.7957456]
DEBUG:chainerrl.agents.ddpg

DEBUG:chainerrl.explorers.additive_ou:t:99 noise:[-0.6091668]
DEBUG:chainerrl.explorers.additive_ou:t:99 noise:[-0.17506137]
DEBUG:chainerrl.explorers.additive_ou:t:99 noise:[-0.08720013]
DEBUG:chainerrl.explorers.additive_ou:t:99 noise:[-0.08819804]
DEBUG:chainerrl.agents.ddpg:t:104 a:[0.43038756] q:[[-0.18314293]
 [-0.27171105]
 [-0.13378528]]
DEBUG:chainerrl.explorers.additive_ou:t:104 noise:[-0.5174932]
DEBUG:chainerrl.explorers.additive_ou:t:104 noise:[-0.29701158]
DEBUG:chainerrl.explorers.additive_ou:t:104 noise:[-0.12568554]
DEBUG:chainerrl.explorers.additive_ou:t:104 noise:[0.1807302]
DEBUG:chainerrl.explorers.additive_ou:t:104 noise:[-0.39907402]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.1798389] q:[[-0.30488625]
 [-0.288438  ]
 [-0.14175686]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.17919189] q:[[-0.30836287]
 [-0.29141337]
 [-0.13874263]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.47068942] q:[[-0.21922004]
 [-0.29468518]
 [-0.13471822]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.180873

DEBUG:chainerrl.agents.ddpg:t:108 a:[0.04867048] q:[[-0.06283718]
 [-0.00108431]
 [-0.05931465]
 [-0.15624341]
 [-0.32123643]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.05277245] q:[[-0.06750035]
 [-0.0011677 ]
 [-0.03201357]
 [-0.15775424]
 [-0.32182834]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.05780423] q:[[-0.07204778]
 [-0.00148492]
 [-0.01302674]
 [-0.15981624]
 [-0.14087494]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28823328] q:[[-0.07492077]
 [-0.00132803]
 [-0.00405078]
 [-0.16383485]
 [-0.14113908]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2874484] q:[[-0.0757818 ]
 [-0.00108529]
 [-0.00080694]
 [-0.17270374]
 [-0.14138627]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28650233] q:[[-0.07664032]
 [-0.00100577]
 [-0.00083222]
 [-0.18611681]
 [-0.1418195 ]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28508115] q:[[-0.07791959]
 [-0.00109115]
 [-0.00083222]
 [-0.19894904]
 [-0.14232858]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2830979] q:[[-0.0791833 ]
 [-0.00134588]
 [-0.00083222]
 [-0.19773895]
 [

DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28298426] q:[[-0.07924263]
 [-0.00277067]
 [-0.00083223]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28001145] q:[[-0.07999642]
 [-0.00299769]
 [-0.00083223]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2778537] q:[[-0.08076458]
 [-0.00322504]
 [-0.00083223]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27623063] q:[[-0.08156424]
 [-0.00343993]
 [-0.00083223]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27496508] q:[[-0.08220026]
 [-0.0036502 ]
 [-0.00083223]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.274867] q:[[-0.08290179]
 [-0.00386625]
 [-0.00083223]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27552032] q:[[-0.08403297]
 [-0.00409616]
 [-0.00083223]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:cha

DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2759996] q:[[-0.08132064]
 [-0.00676906]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2752278] q:[[-0.08201931]
 [-0.00722216]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27500612] q:[[-0.08264118]
 [-0.00757266]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27552685] q:[[-0.08373098]
 [-0.00781655]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27638724] q:[[-0.08480953]
 [-0.00802398]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27724802] q:[[-0.08606513]
 [-0.00815899]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27827477] q:[[-0.08719576]
 [-0.00818594]
 [-0.00083321]
 [-0.00083493]
 [-0.00083814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27976757] q:[[-0.08932602]
 [-0.00819036]
 [-0.00083321]
 [-0.00083493]
 [

...

DEBUG:chainerrl.agents.ddpg:t:108 a:[0.12835562] q:[[-0.05235668]
 [-0.00956556]
 [-0.00096512]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27679956] q:[[-0.08561197]
 [-0.00987138]
 [-0.00094985]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27819252] q:[[-0.08695244]
 [-0.01019493]
 [-0.00093852]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27947497] q:[[-0.08892918]
 [-0.01053715]
 [-0.00092848]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28131258] q:[[-0.09112474]
 [-0.01090951]
 [-0.0009157 ]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2831403] q:[[-0.09374436]
 [-0.01131489]
 [-0.00090254]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28501678] q:[[-0.09604944]
 [-0.01173675]
 [-0.0009027 ]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28669903] q:[[-0.09711316]
 [-0.01217454]
 [-0.0009067 ]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28836834] q:[[-0.09794497]
 [-0.01262231]
 [-0.00093896]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28986865] q:[[-0.09862334]
 [-0.01308168]
 [-0.00100913]]
DEBUG:chainerrl.agents.ddpg:t:1

DEBUG:chainerrl.agents.ddpg:t:108 a:[0.46889985] q:[[-0.21649948]
 [-0.14947252]
 [-0.15035094]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.16865402] q:[[-0.1511893 ]
 [-0.15134597]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.17112142] q:[[-0.15294649]
 [-0.1523543 ]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.17359494] q:[[-0.15462793]
 [-0.15336272]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.17609744] q:[[-0.15633559]
 [-0.15438211]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.17864683] q:[[-0.15809849]
 [-0.15541102]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.18124804] q:[[-0.15987484]
 [-0.15644282]
 [-0.320785  ]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.18392786] q:[[-0.16164416]
 [-0.15747835]
 [-0.32166445]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.18668467] q:[[-0.16340855]
 [-0.15851891]
 [-0.14001673]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.18949637] q:[[-0.16517328]
 [-0.15956919]
 [-0.14037691]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.19238918] q:[[-0.16692454]
 [-0.16059801]
 [-0.14067563]]
DEBUG:ch

DEBUG:chainerrl.agents.ddpg:t:108 a:[0.43267348] q:[[-0.18987027]
 [-0.15836713]
 [-0.15180564]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.43705374] q:[[-0.1934699 ]
 [-0.1596324 ]
 [-0.15296148]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.45509195] q:[[-0.20327461]
 [-0.16093412]
 [-0.1541352 ]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.45819384] q:[[-0.20621425]
 [-0.16224858]
 [-0.15532327]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.46108288] q:[[-0.20902048]
 [-0.16355142]
 [-0.15652338]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.4634781] q:[[-0.21158397]
 [-0.16493194]
 [-0.15773207]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.15146421] q:[[-0.16630891]
 [-0.15894395]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.15373051] q:[[-0.16767466]
 [-0.16016391]
 [-0.32133377]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.15606306] q:[[-0.16903563]
 [-0.16139002]
 [-0.13985248]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.15827455] q:[[-0.17062359]
 [-0.16262333]
 [-0.13781814]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.1597835

DEBUG:chainerrl.agents.ddpg:t:108 a:[0.27817774] q:[[-0.0869939 ]
 [-0.05087446]
 [-0.17555311]
 [-0.14376865]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2795025] q:[[-0.08899136]
 [-0.04768819]
 [-0.19047922]
 [-0.14474751]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28136376] q:[[-0.09121684]
 [-0.04570948]
 [-0.2023415 ]
 [-0.14573435]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2831992] q:[[-0.09383994]
 [-0.04450822]
 [-0.21191227]
 [-0.14674583]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28508765] q:[[-0.09610371]
 [-0.04392996]
 [-0.21737728]
 [-0.14776322]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2867627] q:[[-0.09711532]
 [-0.04381057]
 [-0.22516866]
 [-0.14878881]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2884339] q:[[-0.09795047]
 [-0.04357282]
 [-0.2237691 ]
 [-0.14979291]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.28992325] q:[[-0.09863135]
 [-0.04325772]
 [-0.21352068]
 [-0.15080822]]
DEBUG:chainerrl.agents.ddpg:t:108 a:[0.2907198] q:[[-0.09933148]
 [-0.04265204]
 [-0.19644164]
 [-0.15183945

AssertionError: This env is already closed

1. experiments.train_agent_batch_with_evaluation()

2. contents in train_agent_batch

    1. actions = agent.batch_act_and_train(obss)
    2. env.step(actions)
    3. compute resets
    4. agent.batch_observe_and_train(obss, rs, dones, resets): in this env, done==True when horizon is reached. So resets is same as dones

In [44]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
recent_returns = deque(maxlen=return_window_size)

num_envs = env.num_envs
episode_r = np.zeros(num_envs, dtype=np.float64)
episode_idx = np.zeros(num_envs, dtype='i')
episode_len = np.zeros(num_envs, dtype='i')

# o_0, r_0
obss = env.reset()
rs = np.zeros(num_envs, dtype='f')

t = step_offset

In [14]:
for _ in range(5):
    # a_t
    actions = agent.batch_act_and_train(obss)
    # o_{t+1}, r_{t+1}
    obss, rs, dones, infos = env.step(actions)
    episode_r += rs
    episode_len += 1

    # mask for reset the env(when collision or horizon)
    env_reset = dones
    not_env_reset = np.logical_not(env_reset)  # doesn't reset when True

    # Agent observes the consequences
    agent.batch_observe_and_train(obss, rs, dones, env_reset)

    episode_idx += env_reset
    recent_returns.extend(episode_r[env_reset])

    for _ in range(num_envs):
        t += 1

    # logger should be here
    if (log_interval is not None
            and t >= log_interval
            and t % log_interval < num_envs):
        logger.info(
            'outdir:{} step:{} episode:{} last_R: {} average_R:{}'.format(  # NOQA
                outdir,
                t,
                np.sum(episode_idx),
                recent_returns[-1] if recent_returns else np.nan,
                np.mean(recent_returns) if recent_returns else np.nan,
            ))
        logger.info('statistics: {}'.format(agent.get_statistics()))

    # evaluator should be here

    if t >= steps:
        break

    # Start new episodes if needed
    episode_r[env_reset] = 0
    episode_len[env_reset] = 0
    obss = env.reset(not_env_reset)

DEBUG:chainerrl.agents.ddpg:t:0 a:[0.17891979] q:[[-0.37691563]
 [-0.29338697]
 [-0.1590812 ]]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.5457015]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.2952096]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.6980542]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.1124804]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[0.69293666]
DEBUG:chainerrl.agents.ddpg:t:4 a:[0.09080206] q:[[-0.29664254]
 [-0.1683121 ]]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[1.1955103]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[2.1975727]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[1.6391486]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[1.3692688]
DEBUG:chainerrl.agents.ddpg:t:8 a:[0.09270164] q:[[-0.29954857]
 [-0.17745703]]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[1.4674435]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[1.4067122]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[0.76240957]
DEBUG:chainerrl.explore

# For not native multi-merge env

In [ ]:
class DDPG_MA(ddpg.DDPG):
    def batch_act(self, batch_obs):
        """Select a batch of actions for evaluation.
        Args:
            batch_obs (Sequence of ~object): Observations.
        Returns:
            Sequence of ~object: Actions.
        """

        batch_actions = []
        for env_obs in batch_obs:
            keys, obss = list(env_obs.keys()), list(env_obs.values())
            obss = [obs.astype(np.float32) for obs in obss]
            
            with chainer.using_config('train', False), chainer.no_backprop_mode():
                batch_xs = self.batch_states(obss, self.xp, self.phi)
                batch_action = self.policy(batch_xs).sample()
                # Q is not needed here, but log it just for information
                q = self.q_function(batch_xs, batch_action)

            batch_actions.append({key:cuda.to_cpu(action.array) for key, action in zip(keys, batch_action)})
            
            # Update stats
            self.average_q *= self.average_q_decay
            self.average_q += (1 - self.average_q_decay) * float(
                q.array.mean(axis=0))
            
        self.logger.debug('t:%s a:%s q:%s',
                          self.t, batch_action.array[0], q.array)
        return batch_actions

    def batch_act_and_train(self, batch_obs):
        """Select a batch of actions for training.
        Args:
            batch_obs (Sequence of ~object): Observations.
        Returns:
            Sequence of ~object: Actions.
        """

        batch_greedy_action = self.batch_act(batch_obs)
        batch_action = [
            {key: self.explorer.select_action(self.t, lambda: action) for key, action in actions.items()}
            for actions in batch_greedy_action] 

        self.batch_last_obs = list(batch_obs)
        self.batch_last_action = list(batch_action)
        
        return batch_action

    def batch_observe_and_train(
            self, batch_obs, batch_reward, batch_done, batch_reset):
        """Observe a batch of action consequences for training.
        Args:
            batch_obs (Sequence of ~object): Observations.
            batch_reward (Sequence of float): Rewards.
            batch_done (Sequence of boolean): Boolean values where True
                indicates the current state is terminal.
            batch_reset (Sequence of boolean): Boolean values where True
                indicates the current episode will be reset, even if the
                current state is not terminal.
        Returns:
            None
        """
        
        for i in range(len(batch_obs)):
            if self.batch_last_obs[i] is not None:
                assert self.batch_last_action[i] is not None           
                for key in self.batch_last_obs[i].keys():
                    if key in batch_obs[i]:
                        self.t += 1
                        # Update the target network
                        if self.t % self.target_update_interval == 0:
                            self.sync_target_network()
                        # Add a transition to the replay buffer
                        self.replay_buffer.append(
                            state=self.batch_last_obs[i][key],
                            action=self.batch_last_action[i][key],
                            reward=batch_reward[i][key],
                            next_state=batch_obs[i][key],
                            next_action=None,
                            is_state_terminal=batch_done[i][key]
                        )
                        self.replay_updater.update_if_necessary(self.t)
                # set None when env is beggining
                if batch_reset[i]:
                    self.batch_last_obs[i] = None

In [14]:
for _ in range(5):
    # a_t
    actions = agent.batch_act_and_train(obss)
    # o_{t+1}, r_{t+1}
    obss, rs, dones, infos = env.step(actions)
    episode_r += [np.mean(list(rs[i].values())) for i in range(len(rs))]
    episode_len += 1

    # mask for reset the env(when collision or horizon)
    env_reset = [done["__all__"] for done in dones]
    not_env_reset = np.logical_not(env_reset)  # doesn't reset when True

    # Agent observes the consequences
    agent.batch_observe_and_train(obss, rs, dones, env_reset)

    episode_idx += env_reset
    recent_returns.extend(episode_r[env_reset])

    for _ in range(num_envs):
        t += 1

    # logger should be here
    if (log_interval is not None
            and t >= log_interval
            and t % log_interval < num_envs):
        logger.info(
            'outdir:{} step:{} episode:{} last_R: {} average_R:{}'.format(  # NOQA
                outdir,
                t,
                np.sum(episode_idx),
                recent_returns[-1] if recent_returns else np.nan,
                np.mean(recent_returns) if recent_returns else np.nan,
            ))
        logger.info('statistics: {}'.format(agent.get_statistics()))

    # evaluator should be here

    if t >= steps:
        break

    # Start new episodes if needed
    episode_r[env_reset] = 0
    episode_len[env_reset] = 0
    obss = env.reset(not_env_reset)

DEBUG:chainerrl.agents.ddpg:t:0 a:[0.08707273] q:[[-0.2751863 ]
 [-0.15872331]]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.5457015]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.2952096]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.6980542]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.1124804]


AttributeError: 'numpy.float64' object has no attribute 'values'